# Análise de desempenho - Super LP SP

## Geral

In [1]:
# Importing libraries
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
from google.auth import app_engine
from math import ceil
from os import path
import pandas as pd
import datetime
import calendar
import os
import numpy as np
import requests
import json
from google.cloud import bigquery
from google.auth import app_engine
import hashlib
import hmac
import locale

In [2]:
# Defining global vars
secret = bytes('rQt8FyPbHiVAKok2Phkqf8rTzuDDkIfyU8VlUdFo6vJXVU06188rXZTXMkePG8oxBb4zgMYnvYju8cEa8giujeBeyGuyjRcbhmIhtFtJ4eyyH8J0H1j4ujgedwsr7kl2IXafVpfCrXZp8066GiZJnpouSFpK1pNietbBjbbF1di4wtphPgeZZRJwXkAiHArso64wCkmvXz5WAbCaHe5ZS5RkKCxy1fGa03WzdahIQSO1zlpPGrRygPtY8snb0PsJ', 'utf-8')
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = './contabilizei-dataflow-c09ddfe8334e.json'
KEY_FILE_BIGQUERY = './contabilizei-dataflow-c49121aa9533.json'
VIEW_ID = '176785806'
pd.set_option('display.float_format', lambda x: '%.3f' % x)
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [3]:
def email_to_uid(email):
    hash = hmac.new(secret,bytes(email, 'utf-8'),hashlib.sha256)
    return hash.hexdigest()

In [4]:
# Config GA Connection
credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
analytics = build('analyticsreporting', 'v4', credentials=credentials)

In [4]:
# GA Default functions
def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    # Build the service object.
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics

def get_report(analytics, dateRanges, metrics, dimensions, dimensionFilter=[]):
    return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'samplingLevel': 'LARGE',
          'pageSize': 100000,
          'dateRanges': dateRanges,
          'metrics': metrics,
          'dimensions': dimensions,
          'segments': segments,
          'dimensionFilterClauses': dimensionFilter
        }]
      }
  ).execute()


def print_response(response):
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                print(header + ': ' + dimension)

            for i, values in enumerate(dateRangeValues):
                print('Date range: ' + str(i))
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                    print(metricHeader.get('name') + ': ' + value)

def response_to_df(response):
    list = []
    # get report data
    for report in response.get('reports', []):
        # set column headers
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
    
        for row in rows:
            # create dict for each row
            dict = {}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            # fill dict with dimension header (key) and dimension value (value)
            for header, dimension in zip(dimensionHeaders, dimensions):
                dict[header] = dimension

            # fill dict with metric header (key) and metric value (value)
            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    #set int as int, float a float
                    if ',' in value or '.' in value:
                        dict[metric.get('name')] = float(value)
                    else:
                        dict[metric.get('name')] = int(value)

            list.append(dict)
    
    df = pd.DataFrame(list)
    return df

## Dados agrupados

### Getting data

#### Google Analytics

In [5]:
analytics = initialize_analyticsreporting()

##### Loading up-to-date data

In [6]:
metrics = [{'expression': 'ga:users'},
           {'expression': 'ga:newUsers'},
           {'expression': 'ga:sessions'},
           {'expression': 'ga:transactions'},
           {'expression': 'ga:transactionRevenue'},
           {'expression': 'ga:goal10Completions'},
           {'expression': 'ga:goal14Completions'}]

dimensions = [{'name': 'ga:date'},
              {'name': 'ga:experimentCombination'},
              {'name': 'ga:deviceCategory'},
              {'name': 'ga:sourceMedium'},
              {'name': 'ga:dimension1'}, 
              {'name': 'ga:dimension20'}]

segments =  []

dimensionFilters = [{'filters':
                     [{'dimensionName': 'ga:experimentName',
                       'not': False ,
                       'operator': 'IN_LIST',
                       'expressions': ['Super LP SP (Novembro)'],
                       'caseSensitive': False
                     }]
                  }]

In [7]:
def get_ga_single_date(date_to_fetch):
    dateRanges = [{'startDate': date_to_fetch, 'endDate': date_to_fetch}]
    response = get_report(analytics, dateRanges, metrics, dimensions, dimensionFilters)
    return response_to_df(response)
    #return enhance_df_ga(response_to_df(response))

In [8]:
def get_ga_from_date_range(dt_start, dt_end):
    df = pd.DataFrame()
    start = datetime.datetime.strptime(dt_start, "%Y-%m-%d")
    end = datetime.datetime.strptime(dt_end, "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    
    for date in date_generated:
        print(date)
        df = df.append(get_ga_single_date(date.strftime("%Y-%m-%d")), ignore_index=True)

    return df

### Cookies do teste

In [11]:
#dt_inicio = datetime.datetime(2020,10,8,0,0,0)
dt_inicio = datetime.datetime(2020,11,16,0,0,0)
dt_fim = datetime.datetime(2020,12,2,0,0,0)
dt_start_test = str(dt_inicio.date().isoformat())
dt_end_test = str(dt_fim.date().isoformat())
tstp_inicio = int(dt_inicio.timestamp()*1000)
tstp_fim = int(dt_fim.timestamp()*1000)

In [12]:
df_ga_cookies = get_ga_from_date_range(dt_start_test, dt_end_test)

#Alterando as dimensões para pegar todas visitas
dimensions = [{'name': 'ga:date'},
              {'name': 'ga:experimentCombination'},
              {'name': 'ga:deviceCategory'},
              {'name': 'ga:sourceMedium'}, 
              {'name': 'ga:dimension20'}]

df_ga_non_id = get_ga_from_date_range(dt_start_test, dt_end_test)

2020-11-16 00:00:00
2020-11-17 00:00:00
2020-11-18 00:00:00
2020-11-19 00:00:00
2020-11-20 00:00:00
2020-11-21 00:00:00
2020-11-22 00:00:00
2020-11-23 00:00:00
2020-11-24 00:00:00
2020-11-25 00:00:00
2020-11-26 00:00:00
2020-11-27 00:00:00
2020-11-28 00:00:00
2020-11-29 00:00:00
2020-11-30 00:00:00
2020-12-01 00:00:00
2020-11-16 00:00:00
2020-11-17 00:00:00
2020-11-18 00:00:00
2020-11-19 00:00:00
2020-11-20 00:00:00
2020-11-21 00:00:00
2020-11-22 00:00:00
2020-11-23 00:00:00
2020-11-24 00:00:00
2020-11-25 00:00:00
2020-11-26 00:00:00
2020-11-27 00:00:00
2020-11-28 00:00:00
2020-11-29 00:00:00
2020-11-30 00:00:00
2020-12-01 00:00:00


In [13]:
df_ga_cookies = df_ga_cookies.sort_values(by=['ga:dimension1', 'ga:date'])
df_ga = df_ga_cookies.drop_duplicates(subset=['ga:dimension1'], keep='first')

In [14]:
df_ga_cookies['ga:experimentCombination'] = df_ga_cookies['ga:experimentCombination'].replace(['JJ0fFsdXRri7xsQ5thfCQA:0'], 'Controle')
df_ga_cookies['ga:experimentCombination'] = df_ga_cookies['ga:experimentCombination'].replace(['JJ0fFsdXRri7xsQ5thfCQA:1'], 'Variante')

df_ga['ga:experimentCombination'] = df_ga['ga:experimentCombination'].replace(['JJ0fFsdXRri7xsQ5thfCQA:0'], 'Controle')
df_ga['ga:experimentCombination'] = df_ga['ga:experimentCombination'].replace(['JJ0fFsdXRri7xsQ5thfCQA:1'], 'Variante')

df_ga_non_id['ga:experimentCombination'] = df_ga_non_id['ga:experimentCombination'].replace(['JJ0fFsdXRri7xsQ5thfCQA:0'], 'Controle')
df_ga_non_id['ga:experimentCombination'] = df_ga_non_id['ga:experimentCombination'].replace(['JJ0fFsdXRri7xsQ5thfCQA:1'], 'Variante')


In [15]:
print(df_ga_cookies.shape)
print(df_ga_non_id.shape)
print(df_ga.shape)

(2079, 13)
(11568, 12)
(1035, 13)


### Big Query

In [16]:
client = bigquery.Client()

#### Processos

In [17]:
#query_job = client.query("SELECT * FROM dw_views.funil_abertura WHERE dataCadastro between \'" + dt_start_test + "\' AND \'" + dt_end_test + "\' AND arquivado = FALSE")
query_job = client.query("SELECT * FROM `contabilizei-contabilake.vendas.funil_abertura` WHERE dataCadastro >= \'" + dt_start_test +  "\' AND arquivado = FALSE")

In [18]:
df_processos = query_job.result().to_dataframe()

In [18]:
df_processos.shape

(2361, 64)

In [19]:
df_processos['uid'] = df_processos.apply(lambda x: email_to_uid(x['email']), axis=1)

In [20]:
df_merged = pd.merge(df_ga, df_processos, how='left', left_on='ga:dimension1', right_on='uid')
df_merged['Canal'] = 'Online'
df_merged.loc[(df_merged['faturaAdmin'] == True), 'Canal'] = 'Inside Sales'
print(df_merged.shape)

(1111, 79)


In [21]:
df_merged.to_csv('processos-sp-' + str(datetime.datetime.now().timestamp()) + '.csv')

In [22]:
def df_summary(df_raw, df_source):
    # raw: non-id df
    # source: merged ga x internal df
    df1 = df_raw.groupby(by=["ga:experimentCombination"]).sum()[['ga:sessions', 'ga:users']].reset_index()
    df2 = df_source.groupby(by=["ga:experimentCombination"]).count()['ga:dimension1'].reset_index().rename(columns={"ga:dimension1": "ids"})
    df3 = df_source[df_source['uid'].isna() == False].groupby(by=["ga:experimentCombination"]).count()['ga:dimension1'].reset_index().rename(columns={"ga:dimension1": "ids_internos"})
    df4 = df_source[df_source['dataEmissaoFatura'].isna() == False].groupby(by=["ga:experimentCombination"]).count()['ga:dimension1'].reset_index().rename(columns={"ga:dimension1": "faturas"})
    df5 = df_source[df_source['dataEmissaoFatura'].isna() == False].groupby(by=["ga:experimentCombination"]).sum()['valorAbertura'].reset_index().rename(columns={"valorAbertura": "total_faturado"})
    df6 = df_source[df_source['statusPagamento'] == 'CONFIRMADO'].groupby(by=["ga:experimentCombination"]).count()['ga:dimension1'].reset_index().rename(columns={"ga:dimension1": "vendas"})
    df7 = df_source[df_source['statusPagamento'] == 'CONFIRMADO'].groupby(by=["ga:experimentCombination"]).sum()['valorAbertura'].reset_index().rename(columns={"valorAbertura": "total_pago"})
    df8 = df1.merge(df2, on='ga:experimentCombination').merge(df3, on='ga:experimentCombination').merge(df4, on='ga:experimentCombination').merge(df5, on='ga:experimentCombination').merge(df6, on='ga:experimentCombination').merge(df7, on='ga:experimentCombination')
    df8['sessions_by_user'] = df8['ga:sessions'] / df8['ga:users']
    df8['conv_ids'] = df8['ids'] / df8['ga:sessions']
    df8['conv_faturas'] = df8['faturas'] / df8['ga:sessions']
    df8['conv_vendas'] = df8['vendas'] / df8['ga:sessions']
    df8['conv_pagamento'] = df8['vendas'] / df8['faturas']
    df8['valor_medio_faturado'] = df8['total_faturado'] / df8['faturas']
    df8['valor_medio_pago'] = df8['total_pago'] / df8['vendas']
    df8['faturado_por_sessao'] = df8['total_faturado'] / df8['ga:sessions']
    df8['pago_por_sessao'] = df8['total_pago'] / df8['ga:sessions']
    return df8

In [23]:
df_summary = df_summary(df_ga_non_id, df_merged)

In [24]:
df_summary

,ga:experimentCombination,ga:sessions,ga:users,ids,ids_internos,faturas,total_faturado,vendas,total_pago,sessions_by_user,conv_ids,conv_faturas,conv_vendas,conv_pagamento,valor_medio_faturado,valor_medio_pago,faturado_por_sessao,pago_por_sessao
0,kRWNuQiIT0OSim-S9FXiHQ:0,6398,5714,598,266,118,15622.000,102,12698.000,1.120,0.093,0.018,0.016,0.864,132.390,124.490,2.442,1.985
1,kRWNuQiIT0OSim-S9FXiHQ:1,6542,5854,513,168,102,13298.000,91,12099.000,1.118,0.078,0.016,0.014,0.892,130.373,132.956,2.033,1.849


In [25]:
df_merged[df_merged['dataEmissaoFatura'].isna() == False].groupby(by=["ga:experimentCombination", "Canal"]).count()['ga:dimension1'].reset_index()

,ga:experimentCombination,Canal,ga:dimension1
0,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,64
1,kRWNuQiIT0OSim-S9FXiHQ:0,Online,54
2,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,60
3,kRWNuQiIT0OSim-S9FXiHQ:1,Online,42


In [27]:
#df_merged[df_merged['statusPagamento'] == 'CONFIRMADO'].groupby(by=["ga:experimentCombination", "Canal"]).count()['ga:dimension1'].reset_index()
df_merged.groupby(by=["ga:experimentCombination", "statusPagamento"]).count()['ga:dimension1'].reset_index()

,ga:experimentCombination,statusPagamento,ga:dimension1
0,kRWNuQiIT0OSim-S9FXiHQ:0,CONFIRMADO,102
1,kRWNuQiIT0OSim-S9FXiHQ:0,PENDENTE,164
2,kRWNuQiIT0OSim-S9FXiHQ:1,CONFIRMADO,91
3,kRWNuQiIT0OSim-S9FXiHQ:1,PENDENTE,77


In [28]:
df_merged[df_merged['dataEmissaoFatura'].isna() == False].groupby(by=["ga:experimentCombination", "Canal", "formaPagamento"]).count()['ga:dimension1'].reset_index()

,ga:experimentCombination,Canal,formaPagamento,ga:dimension1
0,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,BOLETO,64
1,kRWNuQiIT0OSim-S9FXiHQ:0,Online,BOLETO,18
2,kRWNuQiIT0OSim-S9FXiHQ:0,Online,CARTAO_CREDITO,36
3,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,BOLETO,60
4,kRWNuQiIT0OSim-S9FXiHQ:1,Online,BOLETO,13
5,kRWNuQiIT0OSim-S9FXiHQ:1,Online,CARTAO_CREDITO,29


In [29]:
df_merged[df_merged['statusPagamento'] == 'CONFIRMADO'].groupby(by=["ga:experimentCombination", "Canal", "formaPagamento"]).count()['ga:dimension1'].reset_index()

,ga:experimentCombination,Canal,formaPagamento,ga:dimension1
0,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,BOLETO,54
1,kRWNuQiIT0OSim-S9FXiHQ:0,Online,BOLETO,7
2,kRWNuQiIT0OSim-S9FXiHQ:0,Online,CARTAO_CREDITO,41
3,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,BOLETO,51
4,kRWNuQiIT0OSim-S9FXiHQ:1,Online,BOLETO,9
5,kRWNuQiIT0OSim-S9FXiHQ:1,Online,CARTAO_CREDITO,31


In [30]:
df_merged[df_merged['dataEmissaoFatura'].isna() == False].groupby(by=["ga:experimentCombination", "Canal", "Plano_aquisicao"]).count()['ga:dimension1'].reset_index()

,ga:experimentCombination,Canal,Plano_aquisicao,ga:dimension1
0,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,AVANCADO,23
1,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,BASICO,38
2,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,PREMIUM,3
3,kRWNuQiIT0OSim-S9FXiHQ:0,Online,AVANCADO,7
4,kRWNuQiIT0OSim-S9FXiHQ:0,Online,BASICO,44
5,kRWNuQiIT0OSim-S9FXiHQ:0,Online,PREMIUM,3
6,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,AVANCADO,28
7,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,BASICO,29
8,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,PREMIUM,3
9,kRWNuQiIT0OSim-S9FXiHQ:1,Online,AVANCADO,15


In [38]:
df_merged[df_merged['statusPagamento'] == 'CONFIRMADO'].groupby(by=["ga:experimentCombination", "Canal", "Plano_aquisicao"]).count()['ga:dimension1'].reset_index()

,ga:experimentCombination,Canal,Plano_aquisicao,ga:dimension1
0,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,AVANCADO,16
1,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,BASICO,29
2,kRWNuQiIT0OSim-S9FXiHQ:0,Inside Sales,PREMIUM,2
3,kRWNuQiIT0OSim-S9FXiHQ:0,Online,AVANCADO,5
4,kRWNuQiIT0OSim-S9FXiHQ:0,Online,BASICO,36
5,kRWNuQiIT0OSim-S9FXiHQ:0,Online,PREMIUM,1
6,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,AVANCADO,19
7,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,BASICO,20
8,kRWNuQiIT0OSim-S9FXiHQ:1,Inside Sales,PREMIUM,2
9,kRWNuQiIT0OSim-S9FXiHQ:1,Online,AVANCADO,14


In [39]:
df_merged[df_merged['statusPagamento'] == 'CONFIRMADO'].groupby(by=["ga:experimentCombination", "Plano_aquisicao"]).count()['ga:dimension1'].reset_index()

,ga:experimentCombination,Plano_aquisicao,ga:dimension1
0,kRWNuQiIT0OSim-S9FXiHQ:0,AVANCADO,21
1,kRWNuQiIT0OSim-S9FXiHQ:0,BASICO,65
2,kRWNuQiIT0OSim-S9FXiHQ:0,PREMIUM,3
3,kRWNuQiIT0OSim-S9FXiHQ:1,AVANCADO,33
4,kRWNuQiIT0OSim-S9FXiHQ:1,BASICO,39
5,kRWNuQiIT0OSim-S9FXiHQ:1,PREMIUM,3


In [32]:
credential_path = '/home/yuguro/Downloads/contabilizei-dataflow-c49121aa9533.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credential_path
client = bigquery.Client()

dataset_id = 'hiram'
table_id_flow = 'temp_analise_teste_ab_thiago'
    
dataset_ref = client.dataset(dataset_id)

client.load_table_from_dataframe(df_merged,dataset_ref.table(table_id_flow))

## Hubspot

In [40]:
uids = df_merged[df_merged['dataEmissaoFatura'].isna() == True]['ga:dimension1'].unique()
print(len(uids))

795


In [41]:
def build_hs_contact_payload(dt_inicio, uid, after, properties):
    hs_payload = {
      "filterGroups": [
        {
          "filters": [
            {
              "value": dt_inicio,
              "propertyName": "createdate",
              "operator": "GTE"
            },
            {
              "value": uid,
              "propertyName": "userid",
              "operator": "EQ"
            }
          ]
        }
      ],
      "sorts": [
        "id"
      ],
      "properties": properties,
      "limit": 10,
      "after": after
    }
    return hs_payload

In [42]:
def build_hs_deal_payload(dt_inicio, contactId, after, properties):
    hs_payload = {
      "filterGroups": [
        {
          "filters": [
            {
              "value": dt_inicio,
              "propertyName": "createdate",
              "operator": "GTE"
            },
            {
              "value": contactId,
              "propertyName": "associations.contact",
              "operator": "EQ"
            }
          ]
        }
      ],
      "sorts": [
        "id"
      ],
      "properties": properties,
      "limit": 10,
      "after": after
    }
    return hs_payload

In [43]:
def get_contacts_from_hs(dt_inicio, uid, after, properties):
    hs_payload = build_hs_contact_payload(dt_inicio, uid, after, properties)
    j_payload = str(json.loads(json.dumps(hs_payload))).replace("'", '\"')
    hs_response = requests.request("POST", hs_contact_url, data=j_payload, headers=hs_headers, params=hs_querystring)
    if(hs_response.status_code == 200):
        obj_response = hs_response.json()
        new_after = None
        if('paging' in obj_response):
            new_after = obj_response['paging']['next']['after']
        return obj_response['results'], new_after
    else:
        return None, None

In [44]:
def get_deals_from_hs(dt_inicio, contactId, after, properties):
    hs_payload = build_hs_deal_payload(dt_inicio, contactId, after, properties)
    j_payload = str(json.loads(json.dumps(hs_payload))).replace("'", '\"')
    hs_response = requests.request("POST", hs_deal_url, data=j_payload, headers=hs_headers, params=hs_querystring)
    if(hs_response.status_code == 200):
        obj_response = hs_response.json()
        new_after = None
        if('paging' in obj_response):
            new_after = obj_response['paging']['next']['after']
        return obj_response['results'], new_after
    else:
        return None, None

In [45]:
def get_pipelines_from_hs():
    hs_response = requests.request("GET", hs_pipelines_url, headers=hs_headers, params=hs_querystring)
    if(hs_response.status_code == 200):
        obj_response = hs_response.json()
        new_after = None
        if('paging' in obj_response):
            new_after = obj_response['paging']['next']['after']
        return obj_response['results'], new_after
    else:
        return hs_response

In [48]:
def get_hs_stages_url_from_hs(pipelineId):
    hs_response = requests.request("GET", hs_stages_url.replace("{pipelineId}", pipelineId), headers=hs_headers, params=hs_querystring)
    if(hs_response.status_code == 200):
        obj_response = hs_response.json()
        new_after = None
        if('paging' in obj_response):
            new_after = obj_response['paging']['next']['after']
        return obj_response['results'], new_after
    else:
        return hs_response

In [49]:
df_hs_pipes = pd.DataFrame(columns=['pipelineId', 'pipelineLabel'].append(list(df_hs_pipelines[0][0].keys())))
for item in df_hs_pipelines[0]:
    for stage in item["stages"]:
        stage["pipelineId"] = item["id"]
        stage["pipelineLabel"] = item["label"]
        df_hs_pipes = df_hs_pipes.append(stage, ignore_index=True)
df_hs_pipes['label'] = df_hs_pipes['label'].str.title()
df_hs_pipes['pipelineLabel'] = df_hs_pipes['pipelineLabel'].str.title()

NameError: name 'df_hs_pipelines' is not defined

In [50]:
hs_contact_url = "https://api.hubapi.com/crm/v3/objects/contacts/search"
hs_contact_properties = ["clientid", "createdate", "email", "hs_analytics_source", "hs_analytics_source_data_1", "hs_analytics_source_data_2", "hs_object_id", "lastmodifieddate", "userid"]

hs_deal_url = "https://api.hubapi.com/crm/v3/objects/deals/search"
hs_deal_properties = ["createdate", "email", "hs_object_id", "lastmodifieddate", "dealname", "amount", "closedate", "pipeline", "dealstage", "closed_lost_reason"]

hs_pipelines_url = "https://api.hubapi.com/crm/v3/pipelines/deals"
hs_stages_url = "https://api.hubapi.com/crm/v3/pipelines/objectType/{pipelineId}/stages"

hs_querystring = {"archived":"false", "hapikey":"59898527-f568-4a44-8533-0fac9c566a83"}
hs_headers = {
    'accept': "application/json",
    'content-type': "application/json"
}

In [51]:
df_hs_contatos = pd.DataFrame(columns=hs_contact_properties)
for uid in uids:
    iter_after = 0
    print(uid)
    while (iter_after is not None):
        data, iter_after = get_contacts_from_hs(tstp_inicio, uid, iter_after, hs_contact_properties)
        unrolled_data = [x['properties'] for x in data]
        df_hs_contatos = df_hs_contatos.append(unrolled_data, ignore_index=True)

68212672a9bf1c6dc9346e5801db8aba195f765c0196
23e97af1a3de1f5e8397bd803c8d71f92d709906f9987ea8047a62cb32ed50cc
24451864558fcdfbc2c06a8770b33bf29dee66f5bcbb2f7097c0a9bae3688776
24cb883f160935f02f9ba2ec129043be8d74208863c865e69f1e5cfe316a63d9
24df49e0a104bf5e1a3479887d2e28649bc2a7d7a3f506686ead3ae897a0f913
24e156f96aa016a59c4a6b52dbbe6ac21b89b1f43d9bf7190d11a546fd649a9f
2513697691031268627b96539bd1ee3d00cd9ac7b9108f12ba9ee96cf34a3bb4
2534fb7a1598bc2c529e8306b1d0e301c13dd20b0adcd3eacfd4df3c500ce433
2553cf7e84390e606bce41c287dbf2902b19386901dae79717025aed36c28347
2623d9326080281e00dd34c8ca7792e79c07a73f7ff2a7c0a2e21f0924c091d4
26b5074e830e0de02dc1a5485f598f9f8faa7a5e362aaeed9f6d414fbac14e8b
27113357f1b05729601f13d6f3fb287fb3beccc925ae2b0f90df99d1429c825d
27b6c065fa97960c3ae121ead6c78722207e928d80f10aaf2e902a67fdf58109
283cc2a382bc9a5e9dd529c6f8a115d6f1fb1b81f33472e2ce609f077e27ad5f
28e95aac6b64b42a4ff0d6e40341853ab02ac209d5e1df9ba9c68b5e25d6f91a
297e16e222a4a21997ac19f9b424a59643316bb1d4d85

TypeError: 'NoneType' object is not iterable

In [52]:
len(df_hs_contatos['userid'].unique())

304

In [53]:
df_hs_deals = pd.DataFrame(columns=hs_deal_properties)
for contactId in df_hs['hs_object_id'].tolist():
    iter_after = 0
    print(contactId)
    while (iter_after is not None):
        data, iter_after = get_deals_from_hs(tstp_inicio, contactId, iter_after, hs_deal_properties)
        unrolled_data = [x['properties'] for x in data]
        df_hs_deals = df_hs_deals.append(unrolled_data, ignore_index=True)

NameError: name 'df_hs' is not defined

In [227]:
print(df_hs_deals.shape)
print(len(df_hs_deals['email'].unique()))

(200, 11)
145


In [228]:
df_hs_deals2 = pd.merge(df_hs_deals, df_hs_pipes, how='left', left_on='dealstage', right_on='id')

In [229]:
df_hs_deals2.shape

(200, 20)

In [230]:
df_hs_deals3 = pd.merge(df_hs_deals2, df_hs_contatos, how='left', left_on='email', right_on='email')

In [231]:
df_hs_deals3.shape

(200, 28)

In [268]:
df_hs_deals4 = pd.merge(df_hs_deals3, df_merged, how='inner', left_on='userid', right_on='ga:dimension1')

In [269]:
df_hs_deals4.shape

(217, 104)

In [270]:
df_hs_deals4.groupby(by=["ga:experimentCombination"]).count()['ga:dimension1'].reset_index().sort_values(by=['ga:experimentCombination', 'ga:dimension1'], ascending=False)

,ga:experimentCombination,ga:dimension1
1,Variante,102
0,Controle,115


In [271]:
df_hs_deals4.groupby(by=["ga:experimentCombination", "label"]).count()['ga:dimension1'].reset_index().sort_values(by=['ga:experimentCombination', 'ga:dimension1'], ascending=True)

,ga:experimentCombination,label,ga:dimension1
0,Controle,Cadastro Concluído,2
1,Controle,Cliente Interessado,4
3,Controle,Ganho,6
2,Controle,Disparo Massivo,21
4,Controle,Perdido,82
9,Variante,Proposta,2
10,Variante,Proposta Enviada,2
5,Variante,Cold Call + Email 2,5
6,Variante,Disparo Massivo,7
7,Variante,Ganho,10


In [272]:
df_hs_deals4.groupby(by=["closed_lost_reason", "ga:experimentCombination"]).count()['ga:dimension1'].reset_index().sort_values(by=['closed_lost_reason', 'ga:experimentCombination', 'ga:dimension1'], ascending=True)

,closed_lost_reason,ga:experimentCombination,ga:dimension1
0,Acessou por curiosidade,Controle,2
1,Acessou por curiosidade,Variante,3
2,Boleto - No show,Controle,2
3,CNAE não atendido,Controle,5
4,CNAE não atendido,Variante,2
5,Cidade não atendida,Variante,7
6,Duplicado,Controle,8
7,Duplicado,Variante,30
8,Escolheu outra contabilidade,Controle,1
9,Escolheu outra contabilidade,Variante,3


In [245]:
df_hs_deals4.to_csv('deals-hs-sp-' + str(datetime.datetime.now().timestamp()) + '.csv')